# ROS 2 - Apuntes: Librerías

## 1. Objetivo

Este bloque de tutoriales enseña a **desarrollar código ROS 2 usando bibliotecas de cliente** (client libraries), es decir, las APIs que permiten escribir nodos en lenguajes como C++ (`rclcpp`) o Python (`rclpy`).

Los objetivos principales son:

- Entender qué es una *client library* y su papel en ROS 2.
- Crear y organizar un *workspace* de desarrollo con `colcon`.
- Crear paquetes propios.
- Escribir y ejecutar un ejemplo simple de publisher/subscriber.

En tu contexto docente, esto es el puente entre los conceptos vistos con las herramientas CLI (nodos, topics, servicios) y la implementación de esos conceptos en Python o C++.

---

## 2. Qué es una client library en ROS 2

Una **client library** es la capa de API que permite escribir nodos ROS 2 en un lenguaje de programación concreto.

- Proporciona acceso a conceptos ROS 2 como nodos, topics, servicios, parámetros, tiempo, logging, etc.
- Permite definir publishers, subscribers, servicios y clientes usando código “idiomático” del lenguaje elegido (clases en C++, objetos en Python, etc.).

Ejemplos de client libraries mantenidas oficialmente:

- `rclcpp`: client library para C++.
- `rclpy`: client library para Python.

Ambas se apoyan en una capa común (`rcl` en C) y usan generadores de código (`rosidl`) para manejar los tipos de mensajes de manera consistente entre lenguajes.

---

## 3. Otras client libraries de la comunidad

Además de C++ y Python, la comunidad mantiene client libraries para otros lenguajes:

- C (`rclc`).
- Ada.
- Java / Android.
- .NET (C#, F#, etc.).
- Node.js (`rclnodejs`).
- Rust (`rclrs`).
- Flutter / Dart.

Todas persiguen la misma idea: ofrecer una API idiomática del lenguaje, reutilizando la funcionalidad común de `rcl` y los generadores de código para mensajes.[web:197]

Para tu alumnado del C.E. IA y Big Data, las client libraries relevantes inicialmente serán **Python (`rclpy`)** y, si avanzáis hacia eficiencia y robótica más “hardcore”, **C++ (`rclcpp`)**.

---

## 4. Creación y uso de un workspace con colcon

Los tutoriales de este bloque asumen que se trabaja en un **workspace de ROS 2** gestionado con `colcon`.

### 4.1. Crear el workspace

Estructura típica:

```bash
mkdir -p ros2_ws/src
cd ros2_ws
```

- `ros2_ws` es la carpeta raíz del workspace.
- `src` contiene el código fuente de los paquetes (propios o de terceros).

### 4.2. Clonar paquetes de ejemplo

Para practicar, se suelen clonar repositorios de ejemplo en `src`:

```bash
cd ros2_ws/src
# Ejemplo: clonar ejemplos oficiales (rama de la distro correspondiente)
# git clone https://github.com/ros2/examples src/examples -b <distro>
```
Esto último no es estrictamente necesario. Simplemente debemos crear el directorio `src` obligatoriamente, dentro de nuestro workspace.

Si clonamos, tal como se comenta en el ejemplo, se pobla el workspace con paquetes ya preparados, sobre los que se puede practicar compilación, ejecución de nodos, etc.

### 4.3. Resolver dependencias y construir

De vuelta a la raíz del workspace solo queda construir:

```bash
cd ros2_ws
rosdep install --from-paths src --ignore-src -r -y # no es necesario ejecutar si se parte de cero
colcon build
```

- `rosdep install` resuelve e instala dependencias del sistema (paquetes Debian, etc.).
- `colcon build` compila todos los paquetes del workspace.

Tras compilar, se debe **cargar el overlay** del workspace, que se encuentra dentro de la carpeta '/install':

```bash
source install/setup.bash
```

Esto hace que los ejecutables y recursos del workspace estén disponibles en la terminal.

NOTA:    
Se puede editar el fichero '.bashrc' e incorporar las siguientes líneas para poder ejecutar el 'setup.bash' directamente:   

```bash
nano ~/.bashrc
```   
Una vez tenemos el fichero en edición, colocarnos al final del mismo e incorporar las siguientes líneas:   

```bash
# ROS 2 Kilted (sistema)
source /opt/ros/kilted/setup.bash

# ROS 2 workspace (overlay)
source /ruta_a_tu_workspace/install/setup.bash

```  

---

## 5. Crear un paquete propio

Uno de los pasos clave del bloque es aprender a **crear paquetes** donde vivirá el código de tus nodos (ya sea en C++ o Python).

Un paquete es una parte de una aplicación.
Por ejemplo:  disponemos de un brazo robot que utilizamos para coger y mover objetos. Antes de crear cualquier tipo de `nodo`, se debería dividir la aplicación en varias partes, es decir, en varios paquetes.    

Podemos tener un paquete responsable de la gestión de la cámara, otro para el control de los servos y otro para la gestión de los movimientos del robot.   

Cada paquete es una unidad independiente y responsable de una parte de la aplicación. Para crear un paquete se debe decidir el lenguaje de programación que vayamos a usar. En nuestro caso, usaremos Python como lenguaje de desarrollo, aunque también es posible usar C++.

### 5.1. Requisitos previos

- Tener ROS 2 instalado y el entorno configurado (`source /opt/ros/<distro>/setup.bash`).
- Tener un workspace creado (`ros2_ws` con subcarpeta `src`).
- Estar situado en `ros2_ws/src` antes de crear el paquete.

### 5.2. Crear un paquete estándar

En la terminal:

Para un paquete Python:

```bash
ros2 pkg create --build-type ament_python mi_paquete_py
```   
o, para garantizar todas las dependencias, de esta otra forma:

```bash
ros2 pkg create --build-type ament_python mi_paquete_py --dependencies rclpy
```   


Para un paquete C++:

```bash
ros2 pkg create --build-type ament_cmake mi_paquete_cpp
```
de igual forma, también:

```bash
ros2 pkg create --build-type ament_cmake mi_paquete_cpp --dependencies rclcpp
```

Esto genera la estructura de directorios y archivos base (CMakeLists / setup.py, package.xml, etc.) para comenzar a añadir código.

---

## 6. Ejemplo: Publisher / Subscriber simple (visión general)

Dentro del bloque “Beginner: Client libraries”, un tutorial esencial es **escribir un par de nodos Publisher/Subscriber**.

Objetivo:

- Crear un nodo que publique mensajes de texto en un *topic*.
- Crear otro nodo que se suscriba al mismo topic y muestre esos mensajes.

### 6.1. Flujo general del tutorial (Python)

1. Crear un paquete Python:

```bash
cd ros2_ws/src
ros2 pkg create --build-type ament_python py_pubsub
```

2. Escribir el nodo **publisher** en un fichero Python (por ejemplo `publisher_member_function.py`), usando `rclpy` y publicando mensajes `std_msgs/msg/String` cada cierto tiempo.

3. Escribir el nodo **subscriber** en otro fichero (por ejemplo `subscriber_member_function.py`), que se suscriba al mismo topic y reciba esos mensajes.

4. Declarar ambas entradas en `setup.py` (o `setup.cfg`) dentro de la sección `entry_points` (`console_scripts`).

5. Declarar las dependencias (`rclpy`, `std_msgs`, etc.) en `package.xml`.

6. Volver a la raíz del workspace, construir y cargar el entorno:

```bash
cd ros2_ws
colcon build --packages-select py_pubsub
source install/setup.bash
```

7. Ejecutar nodos (en terminales distintos):   

Un **nodo** es un subprograma de una aplicación, responsable de una sola cosa. Si se dispone de dos funcionalidades diferentes que implementar, entonces existirán dos nodos. Los nodos se comunican entre sí utilizando las comunicaciones ROS 2 (topics, servicios y acciones).

Los nodos se organizan dentro de paquetes. Un paquete, puede tener varios nodos (funcionalidades).
    
Por ejemplo, si retomamos el paquete de gestión de la cámara, se dispondrá de un nodo responsable para el control del hardware de la cámara y, entre otras funcionalidades, enviará imágenes a un nodo de proceso de imagen que, muy probablemente, se encuentre en el mismo paquete de la camara. Este último se encaragara de extraer las coordenadas de los objetos que el robot debe coger.    

Mientras tanto, el nodo encargado de controlar el movimiento del robot, ubicado en el nodo de la gestión de movimientos del robot, calculará los movimientos que debe realizar el robot, ejecutando los comandos específicos y necesarios. 
 A continuación se desarrolla un ejemplo de ejecución de los nodos diseñados:


```bash
ros2 run py_pubsub publisher_member_function
ros2 run py_pubsub subscriber_member_function
```

Resultados esperados: ver cómo el subscriber imprime los mensajes que el publisher envía, reforzando la comprensión de *topics* ahora desde el código.

### 6.2. Conclusiones

Este ejercicio retoma conceptos de:

- Los nodos ahora son programas que el alumnado escribe (no solo ejecutables preinstalados como `turtlesim_node`).
- Los topics que antes inspeccionaban con `ros2 topic list` / `echo` ahora son creados y usados explícitamente en código Python/C++.
- Los alumnos pueden usar la CLI para inspeccionar sus propios nodos: `ros2 node list`, `ros2 topic list`, etc.

---


## Plantilla para NODO en python

In [ ]:
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node


class MyCustomNode(Node): # MODIFY NAME
    def __init__(self):
        super().__init__("node_name") # MODIFY NAME


def main(args=None):
    rclpy.init(args=args)
    node = MyCustomNode() # MODIFY NAME
    rclpy.spin(node)
    rclpy.shutdown()


if __name__ == "__main__":
    main()

## Plantilla para NODO en C++

In [ ]:
#include "rclcpp/rclcpp.hpp"

class MyCustomNode : public rclcpp::Node // MODIFY NAME
{
public:
    MyCustomNode() : Node("node_name") // MODIFY NAME
    {
    }

private:
};

int main(int argc, char **argv)
{
    rclcpp::init(argc, argv);
    auto node = std::make_shared<MyCustomNode>(); // MODIFY NAME
    rclcpp::spin(node);
    rclcpp::shutdown();
    return 0;
}